In [1]:
import numpy as np
from implementations import *
from proj1_helpers import *
import matplotlib.pyplot as plt
from costs import *
from hw_helpers import *
from cross_validation import *
from visualization import *
from gradients import*

In [2]:
def standardize(x):
    #print(np.nanmean(x, axis=0))
    centered_data = np.subtract(x, np.nanmean(x, axis=0))
    std_data = centered_data / np.nanstd(centered_data, axis=0)
    return std_data

In [3]:
def standardize_test(x_tr, x_te):
    mean = np.nanmean(x_tr, axis=0)
    centered_data = np.subtract(x_te, mean)
    std = np.nanstd(x_tr, axis=0)
    std_data = centered_data/std
    return std_data

In [4]:
def penalized_logistic_regression(y, tx, w, lambda_):
    loss = (np.log(1 + np.exp(tx@w))- (y*tx@w) - lambda_/2*(np.linalg.norm(w)*np.linalg.norm(w))).sum()
    gradient = tx.T@(sigmoid(tx@w)-y) - lambda_*w
    #print("gradient", gradient)
    return loss, gradient

In [5]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    loss, gradient = penalized_logistic_regression(y,tx, w, lambda_)
    #print("w", w)
    #print("gg", gradient*gamma)
    w = w-(gradient*gamma)
    return loss,w

In [19]:
def logistic_regression_penalized_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 10000
    gamma = 0.000000001
    lambda_ = 100
    threshold = 1e-8
    losses = []

    # build tx
    w = np.zeros((x.shape[1],1))

    # start the logistic regression
    for it in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, x, w, gamma, lambda_)
        #print(loss.shape, w.shape)
        # log info
        if it % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=it, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent")
    #print("loss={l}".format(l=calculate_loss(y, x, w)))
    return loss, w

In [7]:
yb, input_data, ids = load_csv_data('Data/train.csv')

In [8]:
yb_te, input_data_te, ids_te = load_csv_data('Data/test.csv')

In [9]:
#nan in the last colums are represented by 0
#np.place(input_data[:,29], input_data[:,29] == 0, np.nan)
input_data = np.delete(input_data, [4,5,6,12,23,26,27,28,29],axis= 1)
#np.place(input_data_te[:,29], input_data_te[:,29] == 0, np.nan)
input_data_te = np.delete(input_data_te, [4,5,6,12,23,26,27,28,29],axis= 1)
np.place(input_data, input_data < -998, np.nan)
np.place(input_data_te, input_data_te < -999, np.nan)

In [10]:
#standarization
#std_input = standardize(input_data)
#std_input_te = standardize(input_data_te)
#std_input = np.nan_to_num(std_input)
#std_input_te = np.nan_to_num(std_input_te)

In [11]:
yb = yb.reshape(yb.shape[0], -1)

In [12]:
yb_te = yb_te.reshape(yb_te.shape[0], -1)

In [13]:
yb_tr, tx_tr = build_model_data(yb, input_data)

In [14]:
h_split = np.hsplit(tx_tr,tx_tr.shape[1])
degree = 3


In [15]:
poly_tr_tx = h_split[0]
for x in h_split[1:]:
    poly_tr_tx = np.hstack((poly_tr_tx, build_poly(x.ravel(),3)[:,1:]))


In [16]:
poly_tr_tx = standardize(poly_tr_tx)
#std_input_te = standardize(input_data_te)
poly_tr_tx = np.nan_to_num(poly_tr_tx)
#std_input_te = np.nan_to_num(std_input_te)
poly_train = poly_tr_tx[:200000]
poly_test = poly_tr_tx[200000:]
y_train = yb[:200000]
y_test = yb[200000:]

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [17]:
poly_train[:,1].mean()

0.00028297685549754444

In [20]:
e, w = logistic_regression_penalized_gradient_descent_demo(y_train,poly_train)
print(e)

Current iteration=0, loss=138629.43611198905
Current iteration=100, loss=134681.9674584266
Current iteration=200, loss=127028.77705838735
Current iteration=300, loss=115893.85666799589
Current iteration=400, loss=101483.90832085184
Current iteration=500, loss=83989.37622781296
Current iteration=600, loss=63585.634478318774
Current iteration=700, loss=40434.20962711259
Current iteration=800, loss=14683.97663556238
Current iteration=900, loss=-13527.708454571946
Current iteration=1000, loss=-44073.95789586362
Current iteration=1100, loss=-76837.39048431875
Current iteration=1200, loss=-111709.25843760002
Current iteration=1300, loss=-148588.65881401207
Current iteration=1400, loss=-187381.8234839504
Current iteration=1500, loss=-228001.48090169736
Current iteration=1600, loss=-270366.28281526145
Current iteration=1700, loss=-314400.28930628486
Current iteration=1800, loss=-360032.5059961049
Current iteration=1900, loss=-407196.4677822988
Current iteration=2000, loss=-455829.8640198242
Cu

In [145]:
loss, gradient = penalized_logistic_regression(poly_train@w,poly_train,w,0)

gradient.shape

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/lucasgauchoux/Documents/MachineLearning/MLrepo/ML_HW/hw_helpers.py:31: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-t))


(45, 1)

In [146]:
test_data = predict_labels(w, poly_test)

In [147]:
false_pred = 0
for i in range(len(test_data)):
    if not test_data[i]==y_test[i]:
        false_pred += 1

In [148]:
false_pred

13826

In [92]:
poly_tr_tx[:,3].mean()

4.4579309133041534e-15

In [ ]:
yb_te, tx_te = build_model_data(yb_te, std_input_te)
h_split = np.hsplit(tx_te,tx_te.shape[1])
degree = 3
poly_tr_te = h_split[0]
for x in h_split[1:]:
    poly_tr_te = np.hstack((poly_tr_te, build_poly(x.ravel(),3)[:,1:]))

In [ ]:
y_pred = predict_labels(w, poly_tr_te)

In [ ]:
create_csv_submission(ids_te, y_pred, 'ridge_poly_constant.csv')

In [53]:
y_pred.shape

(568238, 1)